<a href="https://colab.research.google.com/github/Titaniumexe/Vetinstant-Dog_Cat_Classifier/blob/main/Dog_Cat_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dog-Cat-Classifier**

Unzipping Microsoft Cats Vs. Dogs Dataset

In [ ]:
import os
import shutil
import random
import zipfile

with zipfile.ZipFile('/content/drive/MyDrive/kagglecatsanddogs_5340.zip', 'r') as zip_ref:
       zip_ref.extractall()


Creating Train and Test folders and splitting dataset

In [ ]:
import os
base_dir= "/content/PetImages"
train_dir="/content/train"
test_dir="/content/test"


#Creating Training and Testing Directories if they dont exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for class_name in ['Cat', 'Dog']:
  class_dir=os.path.join(base_dir, class_name)
  image_files=os.listdir(class_dir)
  random.shuffle(image_files)       #Randomly shuffing images into test and train
  split_index= int((0.8)*len(image_files))

  for i, image_file in enumerate(image_files):
    source_path= os.path.join(class_dir, image_file)
    if i<split_index:
      destination_path=os.path.join(train_dir, class_name, image_file)
    else:
      destination_path=os.path.join(test_dir, class_name, image_file)

    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    shutil.copy(source_path, destination_path)


print("Test and Train folders created successfully")



Test and Train folders created successfully


In [ ]:

# Total number of images in Training
print(f"Total number of training Cats is {len(os.listdir('/content/train/Cat'))}")
print(f"Total number of training Dogs is {len(os.listdir('/content/train/Dog'))}")

# Total number of images in Validation
print(f"Total number of validation Cats is {len(os.listdir('/content/test/Cat'))}")
print(f"Total number of validation Dogs is {len(os.listdir('/content/test/Dog'))}")

Total number of training Cats is 10000
Total number of training Dogs is 10000
Total number of validation Cats is 2501
Total number of validation Dogs is 2501


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

Importing ImageNet Pretrained model InceptionV3 taht has 49 layers. Using layer mixed7, as it has maximum features with greater clarity

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2025-05-27 07:54:17--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 142.251.2.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   118MB/s    in 0.7s    

2025-05-27 07:54:18 (118 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(224, 224, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

Setting non-trainable layers in classification level so that weights don't change

In [ ]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=20,
        class_mode='binary')

Found 19998 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.


Preprocessing and augmenting model

In [ ]:

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['acc'])

Obtaining summary of model

In [ ]:
model.build(input_shape=(20, (224,224) ))
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_94 (Conv2D)  │ (None, 111, 111,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 111, 111,  │         96 │ conv2d_94[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_94       │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 109, 109,  │      9,216 │ activation_94[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_95       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_96 (Conv2D)  │ (None, 109, 109,  │     18,432 │ activation_95[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_96[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_96       │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 54, 54,    │          0 │ activation_96[0]… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_97 (Conv2D)  │ (None, 54, 54,    │      5,120 │ max_pooling2d_4[… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_97[0][0]   │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_97       │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_98 (Conv2D)  │ (None, 52, 52,    │    138,240 │ activation_97[0]… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_98[0][0]   │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_98       │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 122,223,521 (466.25 MB)

 Trainable params: 113,248,257 (432.01 MB)

 Non-trainable params: 8,975,264 (34.24 MB)

Training our model with transfer learning

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=20,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=20,
      verbose=2)

Epoch 1/10
20/20 - 151s - 8s/step - acc: 0.6750 - loss: 4.1015 - val_acc: 0.9625 - val_loss: 0.1501
Epoch 2/10
20/20 - 135s - 7s/step - acc: 0.9075 - loss: 0.2973 - val_acc: 0.9625 - val_loss: 0.1468
Epoch 3/10
20/20 - 201s - 10s/step - acc: 0.9325 - loss: 0.1655 - val_acc: 0.9775 - val_loss: 0.0591
Epoch 4/10
20/20 - 142s - 7s/step - acc: 0.9325 - loss: 0.2210 - val_acc: 0.9700 - val_loss: 0.1266
Epoch 5/10
20/20 - 201s - 10s/step - acc: 0.9425 - loss: 0.2050 - val_acc: 0.9750 - val_loss: 0.1424
Epoch 6/10
20/20 - 142s - 7s/step - acc: 0.9600 - loss: 0.1138 - val_acc: 0.9775 - val_loss: 0.0497
Epoch 7/10
20/20 - 143s - 7s/step - acc: 0.9425 - loss: 0.2205 - val_acc: 0.9850 - val_loss: 0.0352
Epoch 8/10
20/20 - 141s - 7s/step - acc: 0.9475 - loss: 0.1768 - val_acc: 0.9775 - val_loss: 0.0976
Epoch 9/10
20/20 - 142s - 7s/step - acc: 0.9575 - loss: 0.0911 - val_acc: 0.9725 - val_loss: 0.0715
Epoch 10/10
20/20 - 142s - 7s/step - acc: 0.9750 - loss: 0.1162 - val_acc: 0.9675 - val_loss: 0.07

Testing our model with input

In [52]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow import expand_dims

img = load_img(
    "/content/test2.jpg", target_size=(224, 224)
)
img_array = img_to_array(img)
img_array = (expand_dims(img_array, 0))/255 # Create a batch
predictions = model.predict(img_array) #Returns probability of class 1:'Dog'
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
[[8.145532e-06]]


In [67]:
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow import expand_dims
import numpy as np

# Load and preprocess the image
img = load_img("/content/test2.jpg", target_size=(224, 224))
img_array = img_to_array(img)
img_array = img_array / 255.0  # Normalize if model expects [0, 1]
img_array = expand_dims(img_array, 0)  # Add batch dimension

# Predict
predictions = model.predict(img_array)

# Interpret predictions
if predictions[0][0]>0.5:
  print("Woof! I'm a dog! With probability  :", predictions[0][0])
else:
  print("Meow! I'm a cat! With probability  :", 1-predictions[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
Meow! I'm a cat! With probability  : 0.99999183
